Previous work of mine and credit goes to already shared in previous notebook

https://www.kaggle.com/code/kumarandatascientist/lb-0-784-efficientnet-b0-pytorch-inference

what changed here 

 HOP_LENGTH = 16
 N_MELS = 148
 FMIN = 20
 FMAX = 16000
   

This notebook is exact copy of the notebook mentioned below but with TTA on Predictions

- [Copied from Notebook](https://www.kaggle.com/code/kumarandatascientist/lb-0-784-efficientnet-b0-pytorch-inference)  

In [1]:
import os
import gc
import warnings
import logging
import time
import math
import cv2
from pathlib import Path
import joblib

import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from soundfile import SoundFile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
import timm
from tqdm.auto import tqdm
from glob import glob
import torchaudio
import random
import itertools
from typing import Union

import concurrent.futures

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

class CFG:
    seed = 42
    print_freq = 100
    num_workers = 2 # CPU 환경에서 너무 많은 워커는 오버헤드를 유발할 수 있으므로 조정

    test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
    submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
    taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv' # 올바른 경로인지 다시 확인

    # 앙상블에 사용할 모든 모델 파일 경로와 해당 아키텍처 이름 (튜플 리스트)
    # 각 튜플은 (모델 파일 경로, 모델 아키텍처 이름) 형식입니다.
    model_configs = [
        # SEResNeXt 모델
        #('/kaggle/input/sedmodel/sedmodel.pth', 'seresnext26t_32x4d'),
        # EfficientNet-B0 모델
        ('/kaggle/input/tf-efficientnet-b1-optuna/model_fold2.pth','tf_efficientnet_b1'),
        ('/kaggle/input/sedmodel/sedmodel.pth','seresnext26t_32x4d')
    ]

    # 오디오 파라미터 (공통)
    SR = 32000
    WINDOW_SIZE = 5 # EfficientNet/ResNet 세그먼트 길이
    target_duration = 5 # SEResNeXt 예측 세그먼트 길이
    train_duration = 10 # SEResNeXt 훈련/컨텍스트 길이
    infer_duration = 5 # SEResNeXt 추론 시 TTA 윈도우

    # 멜 스펙트로그램 파라미터 (일반적인 기본값. 모델 체크포인트 CFG로 오버라이드될 수 있음)
    N_FFT = 1024
    HOP_LENGTH = 512
    N_MELS = 128
    FMIN = 50
    FMAX = 14000
    TARGET_SHAPE = (256, 256) # EfficientNet/ResNet 대상 스펙트로그램 크기
    NORMALIZATION_MODE = 80 # SEResNeXt용 기본 정규화 모드 (top_db 80과 연관)

    in_channels = 1
    device = 'cpu' # CPU로 설정

    # 추론 파라미터
    use_tta = True # EfficientNet에 대한 TTA 활성화 여부
    tta_count = 3 # EfficientNet TTA 변형 수 (원본, 수평 뒤집기, 수직 뒤집기)

    debug = False
    debug_count = 3 # 디버그 시 처리할 파일 개수

cfg = CFG()

print(f"사용 중인 장치: {cfg.device}")
print(f"분류 체계 데이터 로드 중...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"클래스 수: {num_classes}")

def set_seed(seed=42):
    """재현성을 위해 시드를 설정합니다."""
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.seed)

# --- SEResNeXt 모델에 필요한 보조 모듈 (이전 코드에서 가져옴) ---
class AttBlockV2(nn.Module):
    def __init__(self, in_features: int, out_features: int, activation="linear"):
        super().__init__()
        self.activation = activation
        self.att = nn.Conv1d(in_channels=in_features, out_channels=out_features, kernel_size=1, stride=1, padding=0, bias=True)
        self.cla = nn.Conv1d(in_channels=in_features, out_channels=out_features, kernel_size=1, stride=1, padding=0, bias=True)
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.att.weight)
        if hasattr(self.att, "bias") and self.att.bias is not None:
            self.att.bias.data.fill_(0.0)
        nn.init.xavier_uniform_(self.cla.weight)
        if hasattr(self.cla, "bias") and self.cla.bias is not None:
            self.cla.bias.data.fill_(0.0)

    def forward(self, x):
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == "linear":
            return x
        elif self.activation == "sigmoid":
            return torch.sigmoid(x)

# --- 단일 BirdCLEFModel 클래스로 통합 ---
class BirdCLEFModel(nn.Module):
    def __init__(self, model_cfg, num_classes, model_arch_name):
        super().__init__()
        self.model_cfg = model_cfg # 이 모델 인스턴스에 특화된 설정
        self.num_classes = num_classes
        self.model_arch_name = model_arch_name

        self.is_seresnext = "seresnext" in model_arch_name.lower() # 소문자로 변환하여 비교

        if self.is_seresnext:
            self.backbone = timm.create_model(
                model_arch_name,
                pretrained=False,
                in_chans=model_cfg.get('in_channels', 1),
                drop_rate=model_cfg.get('drop_rate', 0.2), # SEResNeXt 훈련 시 드롭아웃
                drop_path_rate=model_cfg.get('drop_path_rate', 0.2),
            )
            layers = list(self.backbone.children())[:-2]
            self.encoder = nn.Sequential(*layers)
            backbone_out = self.backbone.fc.in_features # SEResNeXt의 마지막 FC 레이어 입력 특징 수
            self.fc1 = nn.Linear(backbone_out, backbone_out, bias=True)
            self.att_block = AttBlockV2(backbone_out, self.num_classes, activation="sigmoid")
            self.bn0 = nn.BatchNorm2d(model_cfg.get('n_mels', cfg.N_MELS))

        else: # EfficientNet 또는 ResNet
            self.backbone = timm.create_model(
                model_arch_name,
                pretrained=False,
                in_chans=model_cfg.get('in_channels', 1),
                drop_rate=0.0, # 추론 시 드롭아웃 비활성화
                drop_path_rate=0.0
            )
            if 'efficientnet' in model_arch_name:
                backbone_out = self.backbone.classifier.in_features
                self.backbone.classifier = nn.Identity()
            elif 'resnet' in model_arch_name:
                backbone_out = self.backbone.fc.in_features
                self.backbone.fc = nn.Identity()
            else: # 혹시 모를 다른 모델
                backbone_out = self.backbone.get_classifier().in_features
                self.backbone.reset_classifier(0, '')

            self.pooling = nn.AdaptiveAvgPool2d(1)
            self.classifier = nn.Linear(backbone_out, num_classes)

        # torchaudio 멜 스펙트로그램 변환 (모델별 cfg 사용)
        self.melspec_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=self.model_cfg.get('SR', cfg.SR),
            hop_length=self.model_cfg.get('hop_length', cfg.HOP_LENGTH),
            n_mels=self.model_cfg.get('n_mels', cfg.N_MELS),
            f_min=self.model_cfg.get('f_min', cfg.FMIN),
            f_max=self.model_cfg.get('f_max', cfg.FMAX),
            n_fft=self.model_cfg.get('n_fft', cfg.N_FFT),
            pad_mode="constant",
            norm="slaney",
            onesided=True,
            mel_scale="htk",
        )
        self.db_transform = torchaudio.transforms.AmplitudeToDB(
            stype="power", top_db=model_cfg.get('top_db', 80) # top_db도 model_cfg에서 가져오거나 기본값 사용
        )

    @torch.cuda.amp.autocast(enabled=False)
    def transform_audio_to_spec(self, audio):
        """오디오를 멜 스펙트로그램으로 변환 및 정규화 (torchaudio 사용)"""
        audio = audio.float()
        spec = self.melspec_transform(audio)
        spec = self.db_transform(spec)

        # 정규화 모드 (SEResNeXt 훈련 방식에 따름)
        normal_mode = self.model_cfg.get('normal', cfg.NORMALIZATION_MODE)
        if normal_mode == 80:
            spec = (spec + 80) / 80
        elif normal_mode == 255:
            spec = spec / 255
        else:
            raise NotImplementedError(f"정규화 모드 {normal_mode}는 구현되지 않았습니다.")
        return spec

    def extract_feature_seresnext(self, x_spec):
        """SEResNeXt 모델의 특징 추출 로직"""
        # x_spec: (batch, channels, freq, time)
        # bn0 적용을 위해 (batch, freq, time, channels) 또는 (batch, channels, time, freq)으로 변환
        x = x_spec.permute((0, 1, 3, 2)) # (batch, channel, time, freq)
        
        # bn0는 (N, C, H, W)에서 C에 BatchNorm을 적용하므로, C를 freq (n_mels)로 맞춰야 합니다.
        # 따라서 (batch, freq, time, channel)로 만들고, bn0 적용 후 원래대로 돌려놓습니다.
        x = x.transpose(1, 3) # (batch, freq, time, channel)
        x = self.bn0(x)
        x = x.transpose(1, 3) # (batch, channel, time, freq)

        x = x.transpose(2, 3) # (batch, channel, freq, time) for backbone (encoder)
        x = self.encoder(x) # (batch_size, channels, freq, frames) -> (batch_size, channels, frames)

        x = torch.mean(x, dim=2) # freq 차원 평균 풀링 (batch, channels, frames)

        # 채널 스무딩
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=self.model_cfg.get('drop_rate', 0.5), training=self.training) # dropout rate도 model_cfg에서
        x = x.transpose(1, 2) # (batch, frames, channels)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2) # (batch, channels, frames)
        x = F.dropout(x, p=self.model_cfg.get('drop_rate', 0.5), training=self.training) # dropout rate도 model_cfg에서
        return x

    def attention_infer_seresnext(self, start, end, x_features):
        """SEResNeXt의 어텐션 기반 추론 로직"""
        feat = x_features[:, :, start:end]
        framewise_pred = torch.sigmoid(self.att_block.cla(feat))
        framewise_pred_max = framewise_pred.max(dim=2)[0] # 클립와이즈 예측은 프레임와이즈 최대값
        return framewise_pred_max

    def infer_seresnext(self, x_audio_input, tta_delta=2):
        """SEResNeXt 모델의 추론 메서드 (내부 TTA 로직 포함)"""
        with torch.no_grad():
            x_spec = self.transform_audio_to_spec(x_audio_input)
            x_features = self.extract_feature_seresnext(x_spec)
            time_att = torch.tanh(self.att_block.att(x_features))
            feat_time = x_features.size(-1)

            # 원본 윈도우 예측
            start = (feat_time / 2 - feat_time * (self.model_cfg.get('infer_duration', cfg.infer_duration) / self.model_cfg.get('train_duration', cfg.train_duration)) / 2)
            end = start + feat_time * (self.model_cfg.get('infer_duration', cfg.infer_duration) / self.model_cfg.get('train_duration', cfg.train_duration))
            start = int(start)
            end = int(end)
            pred = self.attention_infer_seresnext(start, end, x_features)

            if cfg.use_tta: # EfficientNet TTA 플래그를 따름 (SEResNeXt도 TTA를 수행하게 함)
                # TTA 시프트
                start_minus = max(0, start - tta_delta)
                end_minus = end - tta_delta
                pred_minus = self.attention_infer_seresnext(start_minus, end_minus, x_features)

                start_plus = start + tta_delta
                end_plus = min(feat_time, end + tta_delta)
                pred_plus = self.attention_infer_seresnext(start_plus, end_plus, x_features)
                final_pred = 0.5 * pred + 0.25 * pred_minus + 0.25 * pred_plus
            else:
                final_pred = pred
            return final_pred

    def forward(self, x_input):
        """
        모델 아키텍처에 따라 포워드 패스를 수행합니다.
        EfficientNet/ResNet: 멜 스펙트로그램(이미지) 입력.
        SEResNeXt: 원본 오디오 입력 (추론 시 infer_seresnext 사용 권장).
        """
        if self.is_seresnext:
            # 이 forward는 훈련 시 사용될 수 있습니다. 추론 시에는 `infer_seresnext`를 직접 호출하세요.
            # 하지만 혹시 이곳으로 들어온다면 (예: 테스트 코드에서 직접 호출)
            # x_input이 원본 오디오라고 가정하고 멜 스펙트로그램 변환을 수행합니다.
            x_spec = self.transform_audio_to_spec(x_input)
            x_features = self.extract_feature_seresnext(x_spec)
            clipwise_output, _, _ = self.att_block(x_features)
            return torch.logit(clipwise_output)
        else: # EfficientNet/ResNet
            features = self.backbone(x_input) # x_input은 이미 멜 스펙트로그램

            if isinstance(features, dict):
                features = features['features']

            if len(features.shape) == 4: # CNN 백본 출력
                features = self.pooling(features)
                features = features.view(features.size(0), -1)
            
            logits = self.classifier(features)
            return logits


# --- 파이프라인 클래스 ---
class BirdCLEF2025Pipeline:
    def __init__(self, cfg):
        self.cfg = cfg
        self.taxonomy_df = None
        self.species_ids = []
        self.models = [] # 모든 모델을 담을 리스트 (SEResNeXt, EfficientNet 등)
        self._load_taxonomy()

    def _load_taxonomy(self):
        print("분류 체계 데이터 로드 중...")
        self.taxonomy_df = pd.read_csv(self.cfg.taxonomy_csv)
        self.species_ids = self.taxonomy_df['primary_label'].tolist()
        print(f"클래스 수: {len(self.species_ids)}")

    def audio2melspec_librosa(self, audio_data, custom_cfg):
        """
        librosa를 사용하여 오디오 데이터를 멜 스펙트로그램으로 변환합니다.
        EfficientNet/ResNet 모델에 사용됩니다.
        """
        if np.isnan(audio_data).any():
            mean_signal = np.nanmean(audio_data)
            audio_data = np.nan_to_num(audio_data, nan=mean_signal)
        
        mel_spec = librosa.feature.melspectrogram(
            y=audio_data,
            sr=custom_cfg.get('SR', self.cfg.SR), # cfg에서 SR 가져오기
            n_fft=custom_cfg.get('N_FFT', self.cfg.N_FFT),
            hop_length=custom_cfg.get('hop_length', self.cfg.HOP_LENGTH),
            n_mels=custom_cfg.get('n_mels', self.cfg.N_MELS),
            fmin=custom_cfg.get('f_min', self.cfg.FMIN),
            fmax=custom_cfg.get('f_max', self.cfg.FMAX),
            power=2.0
        )
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8)
        return mel_spec_norm

    def process_audio_segment_for_cnn_model(self, audio_data, custom_cfg):
        """
        EfficientNet/ResNet과 같은 CNN 모델을 위해 오디오 세그먼트를 멜 스펙트로그램으로 처리합니다.
        """
        segment_len_samples = custom_cfg.get('SR', self.cfg.SR) * custom_cfg.get('WINDOW_SIZE', self.cfg.WINDOW_SIZE)
        if len(audio_data) < segment_len_samples:
            audio_data = np.pad(
                audio_data,
                (0, segment_len_samples - len(audio_data)),
                mode='constant'
            )
            
        mel_spec = self.audio2melspec_librosa(audio_data, custom_cfg)
        
        target_shape = custom_cfg.get('TARGET_SHAPE', self.cfg.TARGET_SHAPE)
        if mel_spec.shape != target_shape:
            # cv2.resize는 (width, height)를 기대하므로, (TARGET_SHAPE[1], TARGET_SHAPE[0]) 순서로 전달
            mel_spec = cv2.resize(mel_spec, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_LINEAR)
            
        return mel_spec.astype(np.float32)

    def apply_tta_cnn(self, spec, tta_idx):
        """EfficientNet/ResNet 모델을 위한 테스트 시간 증강을 적용합니다."""
        if tta_idx == 0:
            return spec
        elif tta_idx == 1: # 수평 뒤집기 (시간 축)
            return np.flip(spec, axis=1).copy()
        elif tta_idx == 2: # 수직 뒤집기 (주파수 축)
            return np.flip(spec, axis=0).copy()
        else:
            return spec

    def find_all_model_files(self):
        """
        `cfg.model_configs`에 명시된 모든 모델 파일 경로를 반환합니다.
        """
        # CFG.model_configs는 (path, name) 튜플 리스트이므로, 경로만 추출합니다.
        return [config[0] for config in self.cfg.model_configs]

    def load_models(self):
        self.models = []
        all_model_configs = self.cfg.model_configs
        
        if not all_model_configs:
            print("경고: 지정된 모델 구성이 없습니다!")
            return self.models

        print(f"지정된 모든 구성에서 총 {len(all_model_configs)}개의 모델 파일을 찾았습니다.")
        
        for model_path, hardcoded_model_arch_name in all_model_configs:
            try:
                print(f"모델 로드 중: {model_path}")
                checkpoint = torch.load(model_path, map_location=torch.device(self.cfg.device))
                
                model_cfg_for_instance = None
                final_model_arch_name = hardcoded_model_arch_name

                if 'cfg' in checkpoint:
                    loaded_cfg_data = checkpoint['cfg']
                    if isinstance(loaded_cfg_data, dict):
                        model_cfg_for_instance = loaded_cfg_data
                    elif hasattr(loaded_cfg_data, '__dict__'):
                        model_cfg_for_instance = loaded_cfg_data.__dict__
                
                if not model_cfg_for_instance:
                    model_cfg_for_instance = self.cfg.__dict__.copy()
                
                model_cfg_for_instance.update({
                    'model_name': final_model_arch_name,
                    'device': self.cfg.device,
                    'in_channels': self.cfg.in_channels,
                    'SR': self.cfg.SR,
                    'target_duration': self.cfg.target_duration,
                    'train_duration': self.cfg.train_duration,
                    'infer_duration': self.cfg.infer_duration,
                    'N_FFT': self.cfg.N_FFT,
                    'HOP_LENGTH': self.cfg.HOP_LENGTH,
                    'N_MELS': self.cfg.N_MELS,
                    'FMIN': self.cfg.FMIN,
                    'FMAX': self.cfg.FMAX,
                    'TARGET_SHAPE': self.cfg.TARGET_SHAPE,
                    'WINDOW_SIZE': self.cfg.WINDOW_SIZE,
                    'NORMALIZATION_MODE': self.cfg.NORMALIZATION_MODE
                })
                
                if 'efficientnet' in final_model_arch_name.lower():
                    model_cfg_for_instance['drop_rate'] = 0.0
                    model_cfg_for_instance['drop_path_rate'] = 0.0


                model = BirdCLEFModel(model_cfg_for_instance, len(self.species_ids), final_model_arch_name)
                
                if 'model_state_dict' not in checkpoint or not checkpoint['model_state_dict']:
                    print(f"경고: {model_path}에서 'model_state_dict'를 찾을 수 없거나 비어 있습니다. 건너뜁니다.")
                    continue

                # --- 핵심 수정: strict=False 옵션 추가 ---
                # SEResNeXt 모델은 melspec_transform 및 db_transform 가중치를 포함할 수 있으므로,
                # SEResNeXt만 strict=True를 유지하고 EfficientNet은 strict=False로 설정하는 것이 더 안전할 수 있습니다.
                # 그러나 둘 다 Missing Keys가 있다면, 모든 모델에 strict=False를 적용합니다.
                if model.is_seresnext: # SEResNeXt 모델인 경우
                    # SEResNeXt 체크포인트가 모든 melspec_transform 키를 가지고 있다면 strict=True
                    # 그렇지 않다면 strict=False
                    # 현재 로그를 보면 EfficientNet에서 Missing keys가 발생했으니, SEResNeXt는 괜찮을 수 있음.
                    # 일단 모든 모델에 strict=False를 적용하여 오류를 해결하고 진행합시다.
                    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
                else: # EfficientNet 또는 ResNet 모델인 경우
                    model.load_state_dict(checkpoint['model_state_dict'], strict=False)

                model = model.to(self.cfg.device)
                model.eval()

                # torchaudio transform은 모델이 로드된 후에 같은 디바이스에 올립니다.
                # 모델 아키텍처에 포함되어 있으므로 이 부분은 유지.
                model.melspec_transform = model.melspec_transform.to(self.cfg.device)
                model.db_transform = model.db_transform.to(self.cfg.device)

                if "seresnext" in final_model_arch_name.lower():
                    model.half().float()

                self.models.append(model)
            except Exception as e:
                print(f"모델 {model_path} ({final_model_arch_name if 'final_model_arch_name' in locals() else 'unknown'}) 로드 오류: {e}")
        
        return self.models

    def predict_single_segment_ensemble(self, audio_data_full, segment_idx, soundscape_id):
        """
        단일 5초 오디오 세그먼트에 대해 모든 앙상블 모델을 사용하여 예측을 수행합니다.
        `run_inference`에서 병렬 처리될 각 오디오 파일에 대해 호출됩니다.
        """
        # 모델별 예측값과 해당 가중치를 저장할 리스트
        individual_model_preds = []
        applied_weights = [] # 각 모델 예측에 적용할 가중치 리스트

        # 5초 창에 대한 세그먼트 경계 계산
        segment_length_samples = self.cfg.SR * self.cfg.WINDOW_SIZE
        start_sample_5sec = segment_idx * segment_length_samples
        end_sample_5sec = min(len(audio_data_full), start_sample_5sec + segment_length_samples)
        current_5sec_audio = audio_data_full[start_sample_5sec:end_sample_5sec]

        # 5초 길이가 안되면 0으로 채움
        if len(current_5sec_audio) < segment_length_samples:
            current_5sec_audio = np.pad(current_5sec_audio,
                                        (0, segment_length_samples - len(current_5sec_audio)),
                                        mode='constant')

        # 모델별 가중치 정의
        # 현재 CFG.model_configs: ('tf_efficientnet_b1'), ('seresnext26t_32x4d')
        # EfficientNetB1 1개와 SEResNeXt 1개에 대한 가중치
        WEIGHT_SERESNEXT = 0.8
        WEIGHT_EFFICIENTNET = 0.2

        for model in self.models:
            model_type = model.model_arch_name.lower()
            
            # 예측값 초기화
            probs = None

            if 'seresnext' in model_type:
                # SEResNeXt 모델은 10초 컨텍스트를 기대하므로 5초 오디오를 10초로 확장합니다.
                expanded_audio_segment = np.zeros(self.cfg.SR * self.cfg.train_duration, dtype=np.float32)
                start_fill_idx = (self.cfg.SR * self.cfg.train_duration - len(current_5sec_audio)) // 2
                expanded_audio_segment[start_fill_idx : start_fill_idx + len(current_5sec_audio)] = current_5sec_audio

                audio_tensor = torch.tensor(expanded_audio_segment, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(self.cfg.device)
                probs = model.infer_seresnext(audio_tensor, tta_delta=2).cpu().numpy().squeeze()
                
                # SEResNeXt에 가중치 할당
                applied_weights.append(WEIGHT_SERESNEXT)
            else: # EfficientNet, ResNet (현재 구성상 EfficientNetB1)
                # EfficientNet/ResNet은 5초 멜 스펙트로그램 입력
                mel_spec = self.process_audio_segment_for_cnn_model(current_5sec_audio, model.model_cfg) # 모델별 cfg 사용

                if self.cfg.use_tta:
                    tta_preds_for_model = []
                    for tta_idx in range(self.cfg.tta_count):
                        augmented_mel_spec = self.apply_tta_cnn(mel_spec, tta_idx)
                        mel_spec_tensor = torch.tensor(augmented_mel_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(self.cfg.device)
                        with torch.no_grad():
                            outputs = model(mel_spec_tensor)
                            probs_tta = torch.sigmoid(outputs).cpu().numpy().squeeze()
                        tta_preds_for_model.append(probs_tta)
                    probs = np.mean(tta_preds_for_model, axis=0) # EfficientNet TTA는 여전히 단순 평균
                else:
                    mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(self.cfg.device)
                    with torch.no_grad():
                        outputs = model(mel_spec_tensor)
                        probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                
                # EfficientNet에 가중치 할당
                applied_weights.append(WEIGHT_EFFICIENTNET)
            
            # 예측값이 계산되었으면 리스트에 추가
            if probs is not None:
                individual_model_preds.append(probs)
            else:
                # 예상치 못한 경우 (모델 타입이 SEResNeXt도 EfficientNet도 아님)
                print(f"경고: 알 수 없는 모델 타입 ({model_type})이 감지되었습니다. 해당 모델의 예측값은 0으로 처리됩니다.")
                individual_model_preds.append(np.zeros(len(self.species_ids)))
                # 가중치 리스트의 길이를 맞추기 위해 0을 추가 (가중치 0과 동일)
                applied_weights.append(0.0) 

        # 모든 앙상블 모델의 예측을 가중 평균
        # np.average 함수를 사용하여 가중치 적용
        if not individual_model_preds:
            # 예측 리스트가 비어있다면, 0으로 채워진 배열 반환 (오류 방지)
            final_ensemble_pred = np.zeros(len(self.species_ids))
        else:
            final_ensemble_pred = np.average(individual_model_preds, axis=0, weights=applied_weights)
        
        end_time_sec = (segment_idx + 1) * self.cfg.WINDOW_SIZE
        row_id = f"{soundscape_id}_{end_time_sec}"
        return row_id, final_ensemble_pred

    def run_inference(self):
        """모든 테스트 사운드스케이프에 대해 앙상블 추론을 실행합니다."""
        test_files = list(Path(self.cfg.test_soundscapes).glob('*.ogg'))
        
        if self.cfg.debug:
            print(f"디버그 모드 활성화됨, {self.cfg.debug_count}개 파일만 사용")
            test_files = test_files[:self.cfg.debug_count]
                
        print(f"총 {len(test_files)}개의 테스트 사운드스케이프를 찾았습니다.")

        all_row_ids = []
        all_predictions = []

        # 각 오디오 파일에 대한 처리 함수
        def process_audio_file(audio_path_str):
            audio_path = Path(audio_path_str)
            soundscape_id = audio_path.stem
            local_row_ids = []
            local_predictions = []
            
            try:
                print(f"Processing {soundscape_id}")
                audio_data_full, _ = librosa.load(audio_path_str, sr=self.cfg.SR, mono=True)
                total_segments = math.ceil(len(audio_data_full) / (self.cfg.SR * self.cfg.WINDOW_SIZE))

                for segment_idx in range(total_segments):
                    row_id, prediction = self.predict_single_segment_ensemble(
                        audio_data_full, segment_idx, soundscape_id
                    )
                    local_row_ids.append(row_id)
                    local_predictions.append(prediction)
            except Exception as e:
                print(f"파일 {audio_path_str} 처리 오류: {e}")
                # 오류 발생 시 해당 사운드스케이프의 모든 세그먼트는 0으로 예측
                # 샘플 제출 파일의 첫 번째 row_id를 복사하여 사용 (길이 맞추기 위함)
                sample_sub = pd.read_csv(self.cfg.submission_csv)
                if not sample_sub.empty:
                    # sample_sub에 있는 row_id 패턴을 활용하여 더미 row_id 생성
                    dummy_row_pattern = f"{soundscape_id}_"
                    # 5초 간격으로 끝나는 row_id를 예상하므로 해당 패턴에 맞춰 생성
                    for seg_idx in range(math.ceil(len(audio_data_full) / (self.cfg.SR * self.cfg.WINDOW_SIZE))):
                        local_row_ids.append(f"{dummy_row_pattern}{(seg_idx + 1) * self.cfg.WINDOW_SIZE}")
                        local_predictions.append(np.zeros(len(self.species_ids)))
                else: # 샘플 제출 파일이 비어있는 경우
                    local_row_ids.append(f"{soundscape_id}_ERROR_0")
                    local_predictions.append(np.zeros(len(self.species_ids)))

            return local_row_ids, local_predictions
        
        # ThreadPoolExecutor를 사용하여 오디오 파일 병렬 처리
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.cfg.num_workers) as executor:
            results = list(tqdm(executor.map(process_audio_file, test_files), total=len(test_files), desc="전체 오디오 파일 처리 중"))

        for rids, preds in results:
            all_row_ids.extend(rids)
            all_predictions.extend(preds)
                
        return all_row_ids, all_predictions

    def create_submission(self, row_ids, predictions):
        """제출 데이터프레임을 생성합니다."""
        print("제출 데이터프레임 생성 중...")
        
        # 예측이 비어있지 않은지 확인
        if not predictions:
            print("생성된 예측값이 없습니다. 샘플 데이터를 사용하여 빈 제출 파일을 생성합니다.")
            submission_df = pd.read_csv(self.cfg.submission_csv)
            submission_df[submission_df.columns[1:]] = 0.0 
            return submission_df
            
        submission_dict = {'row_id': row_ids}
        for i, species in enumerate(self.species_ids):
            submission_dict[species] = [pred[i] for pred in predictions]

        submission_df = pd.DataFrame(submission_dict)
        submission_df.set_index('row_id', inplace=True)

        sample_sub = pd.read_csv(self.cfg.submission_csv, index_col='row_id')
        missing_cols = set(sample_sub.columns) - set(submission_df.columns)
        if missing_cols:
            print(f"경고: 제출 파일에 {len(missing_cols)}개의 종 컬럼이 누락되었습니다.")
            for col in missing_cols:
                submission_df[col] = 0.0

        # 샘플 서브미션의 컬럼 순서와 일치시키기
        submission_df = submission_df[sample_sub.columns]
        submission_df = submission_df.reset_index()
        
        return submission_df

    def smooth_submission(self, submission_path):
        """
        제출 CSV를 후처리하여 예측을 스무딩하여 시간적 일관성을 강화합니다.
        """
        print("제출 예측 스무딩 중...")
        sub = pd.read_csv(submission_path)
        
        if sub.empty:
            print("제출 데이터프레임이 비어 있어 스무딩을 건너뜁니다.")
            return

        cols = sub.columns[1:]
        groups = sub['row_id'].str.rsplit('_', n=1).str[0].values
        unique_groups = np.unique(groups)
        
        for group in unique_groups:
            idx = np.where(groups == group)[0]
            sub_group = sub.iloc[idx].copy()
            predictions = sub_group[cols].values
            new_predictions = predictions.copy()
            
            if predictions.shape[0] > 1:
                # 이웃 세그먼트를 사용하여 예측 스무딩 (SEResNeXt 솔루션의 가중치 사용)
                new_predictions[0] = (predictions[0] * 0.8) + (predictions[1] * 0.2)
                new_predictions[-1] = (predictions[-1] * 0.8) + (predictions[-2] * 0.2)
                for i in range(1, predictions.shape[0]-1):
                    new_predictions[i] = (predictions[i-1] * 0.2) + \
                                         (predictions[i] * 0.6) + \
                                         (predictions[i+1] * 0.2)
            
            sub.iloc[idx, 1:] = new_predictions
        
        sub.to_csv(submission_path, index=False)
        print(f"스무딩된 제출 파일이 {submission_path}에 저장되었습니다.")

    def run(self):
        """전체 추론 파이프라인을 실행하는 메인 메서드입니다."""
        start_time = time.time()
        print("BirdCLEF-2025 추론 시작...")
        print(f"TTA 활성화: {self.cfg.use_tta} (EfficientNet 변형: {self.cfg.tta_count if self.cfg.use_tta else 0}), SEResNeXt는 내부 TTA를 가집니다.")
        
        self.load_models()
        if not self.models:
            print("모델을 찾을 수 없습니다! 모델 경로를 확인해주세요. 종료합니다.")
            # 모델이 없으면 빈 제출 파일 생성
            empty_submission_df = self.create_submission([], [])
            empty_submission_df.to_csv('submission.csv', index=False)
            print("모델을 찾을 수 없어 빈 submission.csv를 생성했습니다.")
            return
            
        print(f"모델 사용: {len(self.models)}개 모델의 앙상블")
        row_ids, predictions = self.run_inference()
        submission_df = self.create_submission(row_ids, predictions)
        
        submission_path = 'submission.csv'
        submission_df.to_csv(submission_path, index=False)
        print(f"초기 제출 파일이 {submission_path}에 저장되었습니다.")
        
        self.smooth_submission(submission_path)
        
        end_time = time.time()
        print(f"추론 완료 시간: {(end_time - start_time) / 60:.2f}분")

# --- 파이프라인 실행 ---
if __name__ == "__main__":
    cfg = CFG()
    print(f"사용 중인 장치: {cfg.device}")
    pipeline = BirdCLEF2025Pipeline(cfg)
    pipeline.run()

사용 중인 장치: cpu
분류 체계 데이터 로드 중...
클래스 수: 206
사용 중인 장치: cpu
분류 체계 데이터 로드 중...
클래스 수: 206
BirdCLEF-2025 추론 시작...
TTA 활성화: True (EfficientNet 변형: 3), SEResNeXt는 내부 TTA를 가집니다.
지정된 모든 구성에서 총 2개의 모델 파일을 찾았습니다.
모델 로드 중: /kaggle/input/tf-efficientnet-b1-optuna/model_fold2.pth
모델 로드 중: /kaggle/input/sedmodel/sedmodel.pth
모델 사용: 2개 모델의 앙상블
총 0개의 테스트 사운드스케이프를 찾았습니다.


전체 오디오 파일 처리 중: 0it [00:00, ?it/s]

제출 데이터프레임 생성 중...
생성된 예측값이 없습니다. 샘플 데이터를 사용하여 빈 제출 파일을 생성합니다.
초기 제출 파일이 submission.csv에 저장되었습니다.
제출 예측 스무딩 중...
스무딩된 제출 파일이 submission.csv에 저장되었습니다.
추론 완료 시간: 0.07분
